# Step One: Data Preparation Tor Dataorep Tabular
Report Text

In this step, the dataset was prepared to ensure suitability for training an XGBoost model. Missing values and duplicate records were removed to improve data quality. The categorical feature was transformed using one-hot encoding to avoid introducing artificial ordinal relationships between categories. Since XGBoost is a tree-based algorithm, feature scaling was not applied. Finally, the dataset was split into training and testing sets for proper evaluation.

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [17]:
# read the csv datafram and show the head
df =pd.read_csv('numerical_tabular.csv')
df.head()

,record_id,date,pharmacy_id,medicine_id,current_stock_level,avg_weekly_sales,reorder_quantity,lead_time_days,supplier_delay_frequency,price_change_rate,storage_capacity,pharmacy_location_code,medicine_category,target_stockout
0,1,2024-03-05,82,46,41,34,45,13,0.80,-0.00,359,9,Painkiller,0
1,2,2024-11-13,42,7,192,13,80,6,0.65,0.13,322,13,Cardiology,0
2,3,2024-11-02,68,26,79,17,62,1,0.55,0.08,242,4,Cardiology,0
3,4,2024-05-06,68,13,180,14,30,14,0.26,0.01,296,8,Antibiotic,0
4,5,2024-03-09,44,28,171,44,71,13,0.58,0.10,101,2,Vitamins,1


In [18]:
# get the tail of the dataframe
df.tail()

,record_id,date,pharmacy_id,medicine_id,current_stock_level,avg_weekly_sales,reorder_quantity,lead_time_days,supplier_delay_frequency,price_change_rate,storage_capacity,pharmacy_location_code,medicine_category,target_stockout
995,996,2024-08-15,69,18,134,32,28,5,0.73,-0.03,340,18,Painkiller,0
996,997,2024-12-28,88,31,140,36,99,3,0.97,0.13,459,17,Diabetes,0
997,998,2024-06-15,60,16,85,30,54,15,0.15,0.06,162,14,Painkiller,0
998,999,2024-03-12,19,39,120,55,88,12,0.08,-0.10,399,14,Cardiology,1
999,1000,2024-03-11,27,10,146,49,25,5,0.19,-0.10,330,13,Antibiotic,0


In [19]:
# get the information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   record_id                 1000 non-null   int64  
 1   date                      1000 non-null   object 
 2   pharmacy_id               1000 non-null   int64  
 3   medicine_id               1000 non-null   int64  
 4   current_stock_level       1000 non-null   int64  
 5   avg_weekly_sales          1000 non-null   int64  
 6   reorder_quantity          1000 non-null   int64  
 7   lead_time_days            1000 non-null   int64  
 8   supplier_delay_frequency  1000 non-null   float64
 9   price_change_rate         1000 non-null   float64
 10  storage_capacity          1000 non-null   int64  
 11  pharmacy_location_code    1000 non-null   int64  
 12  medicine_category         1000 non-null   object 
 13  target_stockout           1000 non-null   int64  
dtypes: float6

In [20]:
# Statistical summary of the dataset
df.describe()

,record_id,pharmacy_id,medicine_id,current_stock_level,avg_weekly_sales,reorder_quantity,lead_time_days,supplier_delay_frequency,price_change_rate,storage_capacity,pharmacy_location_code,target_stockout
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,49.965000,25.395000,98.471000,31.549000,59.276000,7.863000,0.50672,0.047780,296.571000,10.592000,0.262000
std,288.819436,28.070906,14.019755,58.396449,16.264628,23.769535,4.290996,0.27857,0.087536,117.273894,5.730827,0.439943
min,1.000000,1.000000,1.000000,0.000000,5.000000,20.000000,1.000000,0.00000,-0.100000,100.000000,1.000000,0.000000
25%,250.750000,26.000000,13.000000,50.000000,17.000000,38.000000,4.000000,0.26000,-0.022500,196.000000,6.000000,0.000000
50%,500.500000,49.000000,25.000000,95.000000,32.000000,59.000000,8.000000,0.51000,0.040000,293.000000,11.000000,0.000000
75%,750.250000,74.000000,38.000000,149.000000,46.000000,80.000000,11.250000,0.74000,0.130000,398.250000,16.000000,1.000000
max,1000.000000,100.000000,50.000000,200.000000,60.000000,100.000000,15.000000,1.00000,0.200000,500.000000,20.000000,1.000000


In [21]:
# Check for missing values
df.isna().sum()

record_id                   0
date                        0
pharmacy_id                 0
medicine_id                 0
current_stock_level         0
avg_weekly_sales            0
reorder_quantity            0
lead_time_days              0
supplier_delay_frequency    0
price_change_rate           0
storage_capacity            0
pharmacy_location_code      0
medicine_category           0
target_stockout             0
dtype: int64

In [22]:
# Fill missing values with median for numerical columns
df= df.fillna(df.median(numeric_only=True))

In [23]:
# Remove rows with any remaining missing values
df = df.dropna()

In [24]:
# Identify categorical columns
df = df.drop_duplicates()
df.head()

,record_id,date,pharmacy_id,medicine_id,current_stock_level,avg_weekly_sales,reorder_quantity,lead_time_days,supplier_delay_frequency,price_change_rate,storage_capacity,pharmacy_location_code,medicine_category,target_stockout
0,1,2024-03-05,82,46,41,34,45,13,0.80,-0.00,359,9,Painkiller,0
1,2,2024-11-13,42,7,192,13,80,6,0.65,0.13,322,13,Cardiology,0
2,3,2024-11-02,68,26,79,17,62,1,0.55,0.08,242,4,Cardiology,0
3,4,2024-05-06,68,13,180,14,30,14,0.26,0.01,296,8,Antibiotic,0
4,5,2024-03-09,44,28,171,44,71,13,0.58,0.10,101,2,Vitamins,1


In [25]:
df.to_csv("Data_CleanedFor_Tabular.csv")